In [1]:
from importlib.machinery import SourceFileLoader
import pandas as pd
import numpy  as np

wr = SourceFileLoader("mobius", r"../../mobius.py").load_module()
cu = SourceFileLoader("mobius_calib_uncert_lmfit", r"../../mobius_calib_uncert_lmfit.py").load_module()

multi = SourceFileLoader("multiloop", r"../multiloop.py").load_module()
magicopt = SourceFileLoader("magicopt",  r"../magicopt.py").load_module()

In [2]:
wr.initialize('../../../Applications/MAGIC/magic_simple.dll')

parfile = 'templateparameters.dat'
infile  = 'templateinputs.dat'

dataset = wr.DataSet.setup_from_parameter_and_input_files(parfile, infile)

In [3]:
idx = ('Soil', 'Lake')

In [4]:
def do_optim(ds, ID, soil, depo, lake, printout=True) :

    ds.set_parameter_double('Initial exchangeable Ca on soil as % of CEC', ['Soil'], soil['ExCa'])
    ds.set_parameter_double('Initial exchangeable Mg on soil as % of CEC', ['Soil'], soil['ExMg'])
    ds.set_parameter_double('Initial exchangeable Na on soil as % of CEC', ['Soil'], soil['ExNa'])
    ds.set_parameter_double('Initial exchangeable K on soil as % of CEC', ['Soil'], soil['ExK'])
    
    
    params_a, comparisons_a = magicopt.get_acid_anion_setup(ds, idx, 'Observed')
    
    res_pars1, nfev1 = magicopt.calib(ds, params_a, comparisons_a, method='nelder')# method='leastsq')
    
    
    ds.set_parameter_double('Initial exchangeable Ca on soil as % of CEC', ['Soil'], soil['ECa-0'])
    ds.set_parameter_double('Initial exchangeable Mg on soil as % of CEC', ['Soil'], soil['EMg-0'])
    ds.set_parameter_double('Initial exchangeable Na on soil as % of CEC', ['Soil'], soil['ENa-0'])
    ds.set_parameter_double('Initial exchangeable K on soil as % of CEC', ['Soil'], soil['EK-0'])
    
    params, comparisons = magicopt.get_base_cations_setup(ds, 
                            idx, 'Observed', usemax=True, obsE=True)
    
    #params.pretty_print()
    #Set initial guess to the present-day value
    params['ECa_S'].value = soil['ExCa']+20
    params['EMg_S'].value = soil['ExMg']
    params['ENa_S'].value = soil['ExNa']
    params['EK_S'].value = soil['ExK']
    
    params['WCa_S'].value*=0.2
    params['WMg_S'].value*=0.2
    params['WNa_S'].value*=0.2
    params['WK_S'].value*=0.2
    
    params['WK_S'].min = max(0.0, soil['UptK'] - depo['K'])
    
    
    ntries = 0
    err=0
    abserr=0
    
    skip = True
    
    while (abserr>1 and ntries < 10) or ntries==0 :
        if err < 0. :
            params['WCa_S'].value*=2
            if ID==3096:
                params['ECa_S'].value+=5
        elif err > 0.:
            params['WCa_S'].value/=2
            if ID==3096:
                params['ECa_S'].value-=5

        res_pars2, nfev = magicopt.calib(ds, params, comparisons, 
                                    #method='nelder'
                                    method='leastsq'
                                    )

        sim = ds.get_result_series('Ca(2+) ionic concentration', ['Lake'])
        obs = ds.get_input_series('Observed Ca', [], alignwithresults=True)
        err = np.nansum(sim-obs)
        abserr = np.nansum(np.abs(sim-obs))

        ntries += 1
        if skip : break
    
    #if np.abs(err)>1.:
    #    high_error_ids.append((ID,err))
    #    print('High Ca error (abs>1)')

    if printout :
        comp = comparisons
        comp += comparisons_a
        comp += [('SO4(2-) ionic concentration', ['Lake'], 'Observed SO4', [])]

        for comparison in comp :
            simname, simindexes, obsname, obsindexes, *_ = comparison

            sim = ds.get_result_series(simname, simindexes)
            obs = ds.get_input_series(obsname, obsindexes, alignwithresults=True)

            err = np.sqrt(np.nansum(np.square(sim-obs)))

            print('Error of %s: RSSE(sim-obs): %f' % (obsname, err))

        print('\n')
        res_pars1.pretty_print()
        res_pars2.pretty_print()
        print('\n')
        
        ds.write_parameters_to_file('MobiusFiles/params_%d.dat'%ID)
        ds.write_inputs_to_file('MobiusFiles/inputs_%d.dat'%ID)
    
    return res_pars1, res_pars2


def do_fuzzy(ds, ID, soil, depo, lake):
    
    nruns = 10
    
    fuzzy_params = [
        ('Observed NO3', [], 0.9, 1.1, 'obs'),
        ('Observed Ca', [], 0.9, 1.1, 'obs'),
        ('Observed Mg', [], 0.9, 1.1, 'obs'),
        ('Observed Na', [], 0.9, 1.1, 'obs'),
        ('Observed K', [], 0.9, 1.1, 'obs'),
        ('Depth', ['Soil'], 0.9, 1.1, 'par'),
    ]
    
    multipliers = np.zeros((len(fuzzy_params), nruns))
    values      = np.zeros((len(fuzzy_params), nruns))
    
    respars1 = []
    respars2 = []
    
    for idx, par in enumerate(fuzzy_params) :
        name, index, minimum, maximum, partype = par
        multipliers[idx, :] = np.random.uniform(minimum, maximum, nruns)
        
    for run in range(nruns) :
        ds2 = ds.copy()
        for idx, par in enumerate(fuzzy_params):
            name, index, minimum, maximum, partype = par
            if partype == 'obs':
                values = ds2.get_input_series(name, index)
                values *= multipliers[idx, run]
                ds2.set_input_series(name, index, values)
            elif partype == 'par':
                value = ds2.get_parameter_double(name, index)
                value *= multipliers[idx, run]
                ds2.set_parameter_double(name, index, value)
        res1,res2 = do_optim(ds2, ID, soil, depo, lake, printout=True)
        
        respars1.append(res1)
        respars2.append(res2)
        
        ds2.delete()
    
    #TODO: We need to store them somehow instead
    #return fuzzy_params, multipliers, respars1, respars2

In [5]:
#soilfile, lakefile1, lakefile2, depofile, seqfile = excelfiles
excelfiles = ('CCE-soil BJC.xls', 'CCE-lake BJC.xls', 'WRI-lake 2019.xls', 'CCE-depo BJC.xls', 'CCE-seqs BJC.xls')

#multi.do_magic_loop(dataset, excelfile, do_fuzzy, do_id=9)
multi.do_magic_loop(dataset, excelfiles, do_optim, two_year=True, limit_num = 100)#, do_id=9)


#print('IDs with error in Ca > 1: ', high_error_ids)

Running lake Hokksjøen (ID 9)

Error of Observed Ca: RSSE(sim-obs): 0.314168
Error of Observed Mg: RSSE(sim-obs): 1.368018
Error of Observed Na: RSSE(sim-obs): 20.775500
Error of Observed K: RSSE(sim-obs): 0.729593
Error of Observed ECa: RSSE(sim-obs): 0.000000
Error of Observed EMg: RSSE(sim-obs): 0.000000
Error of Observed ENa: RSSE(sim-obs): 0.000000
Error of Observed EK: RSSE(sim-obs): 0.000000
Error of Observed NO3: RSSE(sim-obs): 6.792197
Error of Observed SO4: RSSE(sim-obs): 17.871229


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     26.15     0.01    88.54 4.464e-14     True     None     None
EK_S      5.992     0.01       15 1.062e-14     True     None     None
EMg_S     12.08     0.01    36.41 2.026e-14     True     None     None
ENa_S     5.601     0.01    13.08 1.097e-14     True     None     

Error of Observed Ca: RSSE(sim-obs): 20.323208
Error of Observed Mg: RSSE(sim-obs): 1.541953
Error of Observed Na: RSSE(sim-obs): 8.337397
Error of Observed K: RSSE(sim-obs): 0.010944
Error of Observed ECa: RSSE(sim-obs): 0.000000
Error of Observed EMg: RSSE(sim-obs): 0.000000
Error of Observed ENa: RSSE(sim-obs): 0.000000
Error of Observed EK: RSSE(sim-obs): 0.000000
Error of Observed NO3: RSSE(sim-obs): 0.149870
Error of Observed SO4: RSSE(sim-obs): 7.238160


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     18.27     0.01    84.58 1.088e-13     True     None     None
EK_S      2.437     0.01    9.797 1.49e-14     True     None     None
EMg_S     3.663     0.01    24.07 2.196e-14     True     None     None
ENa_S     1.966     0.01    8.459 1.167e-14     True     None     None
WCa_S     18.93        0    

Error of Observed Ca: RSSE(sim-obs): 11.044768
Error of Observed Mg: RSSE(sim-obs): 1.124561
Error of Observed Na: RSSE(sim-obs): 1.153668
Error of Observed K: RSSE(sim-obs): 0.238704
Error of Observed ECa: RSSE(sim-obs): 0.000004
Error of Observed EMg: RSSE(sim-obs): 0.000001
Error of Observed ENa: RSSE(sim-obs): 0.000000
Error of Observed EK: RSSE(sim-obs): 0.000000
Error of Observed NO3: RSSE(sim-obs): 1.160285
Error of Observed SO4: RSSE(sim-obs): 7.077113


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S      17.5     0.01    73.16 6.74e-06     True     None     None
EK_S       4.32     0.01    13.45 1.786e-06     True     None     None
EMg_S     4.753     0.01    26.15 1.848e-06     True     None     None
ENa_S     1.386     0.01    7.359 5.739e-07     True     None     None
WCa_S     30.81        0    

Error of Observed Ca: RSSE(sim-obs): 2.679697
Error of Observed Mg: RSSE(sim-obs): 0.846645
Error of Observed Na: RSSE(sim-obs): 0.983651
Error of Observed K: RSSE(sim-obs): 1.230644
Error of Observed ECa: RSSE(sim-obs): 0.000006
Error of Observed EMg: RSSE(sim-obs): 0.000000
Error of Observed ENa: RSSE(sim-obs): 0.000000
Error of Observed EK: RSSE(sim-obs): 0.000000
Error of Observed NO3: RSSE(sim-obs): 1.304090
Error of Observed SO4: RSSE(sim-obs): 11.297071


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     32.77     0.01       90 1.004e-05     True     None     None
EK_S        4.1     0.01    12.23 1.404e-06     True     None     None
EMg_S     8.188     0.01    31.63 2.845e-06     True     None     None
ENa_S     4.089     0.01    11.74 1.532e-06     True     None     None
WCa_S     47.39        0   

Error of Observed Ca: RSSE(sim-obs): 24.274460
Error of Observed Mg: RSSE(sim-obs): 8.713448
Error of Observed Na: RSSE(sim-obs): 16.567521
Error of Observed K: RSSE(sim-obs): 2.285541
Error of Observed ECa: RSSE(sim-obs): 0.003058
Error of Observed EMg: RSSE(sim-obs): 0.001643
Error of Observed ENa: RSSE(sim-obs): 0.000521
Error of Observed EK: RSSE(sim-obs): 0.000217
Error of Observed NO3: RSSE(sim-obs): 2.934156
Error of Observed SO4: RSSE(sim-obs): 22.626776


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     29.15     0.01       90  0.02531     True     None     None
EK_S      4.261     0.01    12.59 0.004091     True     None     None
EMg_S     15.74     0.01       50  0.01434     True     None     None
ENa_S     5.324     0.01    14.09 0.005495     True     None     None
WCa_S     52.71        0     

...

Error of Observed Ca: RSSE(sim-obs): 8.781067
Error of Observed Mg: RSSE(sim-obs): 4.089965
Error of Observed Na: RSSE(sim-obs): 2.919892
Error of Observed K: RSSE(sim-obs): 1.226407
Error of Observed ECa: RSSE(sim-obs): 0.061857
Error of Observed EMg: RSSE(sim-obs): 0.032304
Error of Observed ENa: RSSE(sim-obs): 0.010519
Error of Observed EK: RSSE(sim-obs): 0.003212
Error of Observed NO3: RSSE(sim-obs): 1.039835
Error of Observed SO4: RSSE(sim-obs): 2.442241


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     28.21     0.01       90   0.1776     True     None     None
EK_S      2.571     0.01    10.08  0.01632     True     None     None
EMg_S     7.782     0.01    39.54  0.04841     True     None     None
ENa_S     4.335     0.01    13.31  0.02768     True     None     None
WCa_S     25.09        0    187.

Error of Observed Ca: RSSE(sim-obs): 3.971405
Error of Observed Mg: RSSE(sim-obs): 9.743787
Error of Observed Na: RSSE(sim-obs): 25.835641
Error of Observed K: RSSE(sim-obs): 1.053047
Error of Observed ECa: RSSE(sim-obs): 0.000010
Error of Observed EMg: RSSE(sim-obs): 0.000000
Error of Observed ENa: RSSE(sim-obs): 0.000000
Error of Observed EK: RSSE(sim-obs): 0.000001
Error of Observed NO3: RSSE(sim-obs): 1.469367
Error of Observed SO4: RSSE(sim-obs): 3.936951


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     21.45     0.01    82.94 1.426e-05     True     None     None
EK_S       4.02     0.01     12.2 2.828e-06     True     None     None
EMg_S      7.95     0.01    33.48 5.613e-06     True     None     None
ENa_S     5.036     0.01    13.74 3.727e-06     True     None     None
WCa_S     42.93        0   

Error of Observed Ca: RSSE(sim-obs): 92.195843
Error of Observed Mg: RSSE(sim-obs): 8.978662
Error of Observed Na: RSSE(sim-obs): 26.710028
Error of Observed K: RSSE(sim-obs): 6.615413
Error of Observed ECa: RSSE(sim-obs): 0.000102
Error of Observed EMg: RSSE(sim-obs): 0.000647
Error of Observed ENa: RSSE(sim-obs): 0.000434
Error of Observed EK: RSSE(sim-obs): 0.000142
Error of Observed NO3: RSSE(sim-obs): 0.195752
Error of Observed SO4: RSSE(sim-obs): 5.416149


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     14.46     0.01     61.8 0.003615     True     None     None
EK_S       2.69     0.01       10 0.0006946     True     None     None
EMg_S     4.508     0.01     24.4 0.001147     True     None     None
ENa_S     3.733     0.01     11.6 0.001004     True     None     None
WCa_S     69.02        0    1

Error of Observed Ca: RSSE(sim-obs): 11.380176
Error of Observed Mg: RSSE(sim-obs): 0.343612
Error of Observed Na: RSSE(sim-obs): 3.303446
Error of Observed K: RSSE(sim-obs): 0.411679
Error of Observed ECa: RSSE(sim-obs): 0.000266
Error of Observed EMg: RSSE(sim-obs): 0.000036
Error of Observed ENa: RSSE(sim-obs): 0.000069
Error of Observed EK: RSSE(sim-obs): 0.000013
Error of Observed NO3: RSSE(sim-obs): 2.236383
Error of Observed SO4: RSSE(sim-obs): 1.207237


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S      15.9     0.01       73   0.1121     True     None     None
EK_S      2.043     0.01        9  0.01495     True     None     None
EMg_S     1.868     0.01     16.8  0.01324     True     None     None
ENa_S     1.735     0.01      8.2  0.01264     True     None     None
WCa_S     25.81        0    182

Error of Observed Ca: RSSE(sim-obs): 22.306949
Error of Observed Mg: RSSE(sim-obs): 4.712115
Error of Observed Na: RSSE(sim-obs): 3.474753
Error of Observed K: RSSE(sim-obs): 3.833410
Error of Observed ECa: RSSE(sim-obs): 0.010910
Error of Observed EMg: RSSE(sim-obs): 0.006469
Error of Observed ENa: RSSE(sim-obs): 0.002047
Error of Observed EK: RSSE(sim-obs): 0.000637
Error of Observed NO3: RSSE(sim-obs): 32.045177
Error of Observed SO4: RSSE(sim-obs): 20.435566


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     25.26     0.01    84.15  0.05145     True     None     None
EK_S       2.01     0.01    7.946 0.003969     True     None     None
EMg_S        13     0.01    41.88  0.02783     True     None     None
ENa_S     3.975     0.01    10.01 0.008462     True     None     None
WCa_S     50.79        0    2